In [123]:
import tensorflow as tf
import numpy as np
import cv2
import os
import keras
from PIL import Image
from keras.models import model_from_json, load_model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Input
from keras.layers import AveragePooling2D, Flatten, GlobalMaxPooling2D
from keras import layers
from keras.models import Model
config={'黑': 36, 'B': 19, '桂': 45, '2': 62, '陕': 29, '浙': 48, 'N': 3, '1': 58, 'K': 20, 'T': 6, '津': 49, '闽': 44, 'X': 17, '粤': 47, 'Q': 16, 'V': 15, '琼': 41, '皖': 46, '沪': 32, '冀': 52, '鲁': 50, '贵': 35, '川': 31, '吉': 25, '豫': 34, '6': 57, 'L': 21, '5': 63, '晋': 28, '4': 60, 'E': 18, '云': 38, 'S': 7, 'J': 12, 'G': 4, '赣': 30, 'A': 8, 'D': 14, '湘': 40, '鄂': 51, '0': 55, '蒙': 43, 'Y': 22, '辽': 37, 'U': 2, '3': 61, '9': 54, 'W': 24, 'Z': 5, 'P': 23, 'F': 9, 'M': 11, '8': 59, '7': 56, 'R': 1, 'H': 10, '青': 27, 'C': 13, '苏': 33, '甘': 42, '宁': 26, '京': 53, '渝': 39}

In [124]:
def get_data_list(path="车牌字符识别训练数据"):
    fi = []
    rt = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f != '车牌字符识别训练数据' and f != '.DS_Store':
                # 路徑：os.path.join(root, f)
                im = cv2.resize(cv2.imread(os.path.join(root, f)), (24, 48), interpolation=cv2.INTER_CUBIC)
                fi.append(im)
                rt.append(root.split('/')[-1])
                # print(cv2.imread(os.path.join(root, f)))
                # new = fname[0] + 'b' + fname[1]
                # os.rename(os.path.join(rt, f), os.path.join(rt, new))
    print(len(fi))
    return fi, rt


In [125]:
def write_tfrecords(file_name, data_labels, data_datas):
    writer = tf.python_io.TFRecordWriter(file_name + ".tfrecords")
    num = len(data_labels)
    for i in range(num):
        if (i + 1) % 1000 == 0:
            print("以處理{0}數據集{1}張".format(file_name, i + 1))
        label = data_labels[i].encode()
        data = np.resize(data_datas[i], [1, 3456])[0].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(bytes_list=tf.train.BytesList(value=[label])),
            'img_val': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data]))
        }))  # example對象 對label及img_val 進行封裝
        writer.write(example.SerializeToString())
        i += 1
    print("{0}數據集處理完成".format(file_name))
    writer.close()

In [126]:
def print_tfrecord():
    # 將製作好的 tfrecord 數據集文件讀取出來,並轉換成圖片,以驗證數據是否準確無誤
    from PIL import Image

    filename_queue = tf.train.string_input_producer(["train.tfrecords"])  # 讀入數據流
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)  # 返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.string),
                                           'img_val': tf.FixedLenFeature([], tf.string),
                                       })  # 取出包含image和label的feature对象
    image = tf.decode_raw(features['img_val'], tf.uint8)
    label = tf.cast(features['label'], tf.string)

    import matplotlib.pyplot as plt
    % matplotlib inline

    with tf.Session() as sess:  # 開始一個對話
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for i in range(5):
            example, l = sess.run([image, label])  # 在會話中取出image和label數據
            img=np.resize(example, [48, 24, 3])
            plt.title("The picture number is " + str(l.decode('utf-8')))
#             plt.figure(figsize=(2,4))
            plt.imshow(img)
            plt.show()
        coord.request_stop()
        coord.join(threads)
        
# print_tfrecord()

In [127]:
def turn_two_color(name):
    img = cv2.imread(name)
    grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    a = 0
    for i in grayImage:
        b = 0
        for j in i:
            if j < grayImage.mean():  # 比对均值
                grayImage[a][b] = 0
            else:
                grayImage[a][b] = 255
            b += 1
        a += 1
        del b
    cv2.imwrite(name, grayImage)

In [128]:
def make_data():
    from random import shuffle
    filedata, filelabel = get_data_list('车牌字符识别训练数据')
    x = [i for i in range(len(filelabel))] # 18499
    shuffle(x)
    a=[];b=[]
    for i in x[:13000]:
        a.append(filelabel[i])
        b.append(filedata[i])
    write_tfrecords('train',a,b)
    a=[];b=[]
    for i in x[13000:16400]:
        a.append(filelabel[i])
        b.append(filedata[i])
    write_tfrecords('validation',a,b)
    a=[];b=[]
    for i in x[16400:]:
        a.append(filelabel[i])
        b.append(filedata[i])
    write_tfrecords('test',a,b)

In [133]:
def get_tfrecord(filename='train.tfrecords'):
    from PIL import Image

    filename_queue = tf.train.string_input_producer([filename])  # 讀入數據流
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)  # 返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.string),
                                           'img_val': tf.FixedLenFeature([], tf.string),
                                       })  # 取出包含image和label的feature对象
    image = tf.decode_raw(features['img_val'], tf.uint8)
    label = tf.cast(features['label'], tf.string)

    imgdata=[];imglabel=[]
    with tf.Session() as sess:  # 開始一個對話
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for i in range(2):
            example, l = sess.run([image, label])  # 在會話中取出image和label數據
            data = np.resize(example, [48, 24, 3])
            data=turn_two_color(data)
            tem = np.zeros((1, 64))
            tem[0][config[l.decode('utf-8')]] = 1
            imgdata.append(data[0])
            imglabel.append(tem[0])
        coord.request_stop()
        coord.join(threads)
    return imgdata,imglabel


def turn_two_color(data):
    import cv2
    grayImage = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)

    a = 0
    for i in grayImage:
        b = 0
        for j in i:
            if j < grayImage.mean():  # 比对均值
                grayImage[a][b] = 0
            else:
                grayImage[a][b] = 255
            b += 1
        a += 1
        del b
    grayImage=np.resize(grayImage, [1,48, 24])
    return grayImage


In [134]:
def main(train=55000, test=5000, batch_size=1, epochs=2):
    model = create_model()
    tr_data,tr_label=get_tfrecord()
    # from keras.utils import plot_model
    # plot_model(model, to_file='./Resnet_model.png')
    print(tr_data)
    print(tr_label)
#     tem = model.fit(tr_data, tr_label, batch_size=batch_size, epochs=epochs, )
#     print(tem.history)

#     model.evaluate(tsimages, tslabels, batch_size=batch_size)
#     model.save('./resnet.h5')

In [ ]:
main()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import keras
from PIL import Image
from keras.models import model_from_json, load_model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Input
from keras.layers import AveragePooling2D, Flatten, GlobalMaxPooling2D
from keras import layers
from keras.models import Model
config={'黑': 36, 'B': 19, '桂': 45, '2': 62, '陕': 29, '浙': 48, 'N': 3, '1': 58, 'K': 20, 'T': 6, '津': 49, '闽': 44, 'X': 17, '粤': 47, 'Q': 16, 'V': 15, '琼': 41, '皖': 46, '沪': 32, '冀': 52, '鲁': 50, '贵': 35, '川': 31, '吉': 25, '豫': 34, '6': 57, 'L': 21, '5': 63, '晋': 28, '4': 60, 'E': 18, '云': 38, 'S': 7, 'J': 12, 'G': 4, '赣': 30, 'A': 8, 'D': 14, '湘': 40, '鄂': 51, '0': 55, '蒙': 43, 'Y': 22, '辽': 37, 'U': 2, '3': 61, '9': 54, 'W': 24, 'Z': 5, 'P': 23, 'F': 9, 'M': 11, '8': 59, '7': 56, 'R': 1, 'H': 10, '青': 27, 'C': 13, '苏': 33, '甘': 42, '宁': 26, '京': 53, '渝': 39}
config_ = dict(zip(config.values(), config.keys()))
print(config_)

def conv_block(input_tensor, bn_axis, filters, phase, name, strides=(1, 1)):
    """
    Conv2D 塊，雙路雙卷積計算
    :param input_tensor:(tensor) 輸入張量
    :param filters:(tuple) 卷積核打包
    :param strides:(int) 卷積步長
    :param BN_axis:(int) 規範化卷積軸
    :return: model
    """
    filters1, filters2, filters3 = filters  # 解包卷積核數量
    Conv_base_name = 'Conv_' + name + '_' + str(phase) + '_phase_'
    BN_base_name = 'BN_' + name + '_' + str(phase) + '_phase_'
    x = Conv2D(
        filters=filters1, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2a'
    )(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2a')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters2, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2b'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2b')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters3, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2c'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2c')(x)
    x = Activation(activation='relu')(x)

    y = Conv2D(filters3, (1, 1), strides=strides, name=Conv_base_name + '1a')(input_tensor)
    y = BatchNormalization(axis=bn_axis, name=BN_base_name + '1b')(y)

    x = layers.add([x, y])
    a = Activation('relu')(x)

    return a


def identity_block(input_tensor, bn_axis, filters, phase, name, strides=(1, 1)):
    """
    Conv2D 塊，雙路單卷積計算
    :param input_tensor:(tensor) 輸入張量
    :param filters:(tuple) 卷積核打包
    :param strides:(int) 卷積步長
    :param BN_axis:(int) 規範化卷積軸
    :return: model
    """
    filters1, filters2, filters3 = filters  # 解包卷積核數量
    Conv_base_name = 'Conv_' + name + '_' + str(phase) + '_phase_'
    BN_base_name = 'BN_' + name + '_' + str(phase) + '_phase_'
    x = Conv2D(
        filters=filters1, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2a'
    )(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2a')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters2, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2b'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2b')(x)
    x = Activation(activation='relu')(x)

    x = Conv2D(
        filters=filters3, kernel_size=(1, 1), strides=strides, name=Conv_base_name + '2c'
    )(x)
    x = BatchNormalization(axis=bn_axis, name=BN_base_name + '2c')(x)
    x = Activation(activation='relu')(x)

    x = layers.add([x, input_tensor])
    a = Activation('relu')(x)

    return a


def my_resnet():
    inputs = Input(shape=(1, 24, 48))

    x = Conv2D(
        filters=4, kernel_size=(2, 4), padding='same', name='Conv1', data_format='channels_first')(inputs)
    x = BatchNormalization(axis=1, name='BN_Conv1')(x)
    x = Activation('relu')(x)
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format='channels_first')(x)

    x = conv_block(input_tensor=x, bn_axis=1, filters=(4, 4, 64), phase=2, name='a')
    x = identity_block(input_tensor=x, bn_axis=1, filters=(4, 4, 64), phase=2, name='b')
    x = identity_block(input_tensor=x, bn_axis=1, filters=(4, 4, 64), phase=2, name='c')

    # x = conv_block(input_tensor=x, bn_axis=1, filters=(8, 8, 512), phase=3, name='a')
    # x = identity_block(input_tensor=x, bn_axis=1, filters=(8, 8, 512), phase=3, name='b')
    # x = identity_block(input_tensor=x, bn_axis=1, filters=(8, 8, 512), phase=3, name='c')

    x = AveragePooling2D((2, 2), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dense(64, activation='softmax', name='softmax')(x)
#     x = GlobalMaxPooling2D()(x)

    model = Model(inputs, x, name='My_Resnet')
    return model


def create_model():
    """返回一個已創建好的 resnet model"""
    model = my_resnet()
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model




def get_data_list(path="车牌字符识别训练数据"):
    fi = []
    rt = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f != '车牌字符识别训练数据' and f != '.DS_Store':
                # 路徑：os.path.join(root, f)
                im = cv2.resize(cv2.imread(os.path.join(root, f)), (24, 48), interpolation=cv2.INTER_CUBIC)
                fi.append(im)
                rt.append(root.split('/')[-1])
                # print(cv2.imread(os.path.join(root, f)))
                # new = fname[0] + 'b' + fname[1]
                # os.rename(os.path.join(rt, f), os.path.join(rt, new))
    print(len(fi))
    return fi, rt

def get_tfrecord(filename='train.tfrecords',num=1000):
    # 將製作好的 tfrecord 數據集文件讀取出來,並轉換成圖片,以驗證數據是否準確無誤
    from PIL import Image
    print('開始導入數據'+filename)
    filename_queue = tf.train.string_input_producer([filename])  # 讀入數據流
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)  # 返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.string),
                                           'img_val': tf.FixedLenFeature([], tf.string),
                                       })  # 取出包含image和label的feature对象
    image = tf.decode_raw(features['img_val'], tf.uint8)
    label = tf.cast(features['label'], tf.string)

    imgdata=[];imglabel=[]
    with tf.Session() as sess:  # 開始一個對話
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        tem = np.zeros((num, 64))
        for i in range(num):
            if i % 1000==0:
                print('已完成導入數據'+filename+str(i)+'個')
            example, l = sess.run([image, label])  # 在會話中取出image和label數據
            data = np.resize(example, [48, 24, 3])
            data=turn_two_color(data)
            tem[i][config[l.decode('utf-8')]] = 1
            imgdata.append([data])
            imglabel.append(tem[0])
        coord.request_stop()
        coord.join(threads)
    print('導入數據' + filename+'完成')
    return imgdata,tem


def turn_two_color(data):
    import cv2
    grayImage = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    a = 0
    for i in grayImage:
        b = 0
        for j in i:
            if j < grayImage.mean():  # 比对均值
                grayImage[a][b] = 0
            else:
                grayImage[a][b] = 255
            b += 1
        a += 1
        del b
    grayImage=np.resize(grayImage, [24, 48])
    return grayImage

def main(batch_size=100, epochs=10):
    model = create_model()
    tr_data,tr_label=get_tfrecord(num=13000)
    va_data,va_label=get_tfrecord('validation.tfrecords',num=1000)
    print('數據導入完成，開始計算')
    tem = model.fit(tr_data, tr_label, batch_size=batch_size, epochs=epochs, )

    # print(tem.history)
    model.evaluate(va_data, va_label, batch_size=batch_size)
    model.save('./resnet.h5')

# main()


def test_model(num=1000.0):
    ts_images, ts_labels= get_tfrecord('test.tfrecords')

    model = load_model('./resnet.h5')
    pre = model.predict(ts_images)
    tr = 0
    for a, b in zip(ts_labels, pre):
        a = np.argmax(a)
        b = np.argmax(b)
        if a == b:
            tr += 1
        else:
            print('真確'+config_[a])
            print('預測'+config_[b])
    ls = tr / num
    print("正確率為：{0}".format(ls))

test_model()